https://github.com/dwkim78/upsilon

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook 
import numpy as np
from IPython.display import display
from scipy import stats
from collections import OrderedDict
import upsilon

In [2]:
train_ts = pd.read_csv("../data/input/training_set.csv")
display(train_ts.head())

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [3]:
train_meta = pd.read_csv("../data/input/training_set_metadata.csv")
display(train_meta.head())

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,0.017,92
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,0.007,88
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,0.021,42
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,0.007,90
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,0.024,90


In [6]:
sample_data = train_ts.query('object_id == 615 & passband == 5')
date = sample_data["mjd"]
mag = sample_data["flux"]
err = sample_data["flux_err"]

In [9]:
e_features = upsilon.ExtractFeatures(date, mag, err)
e_features.run()
features = e_features.get_features()
features

/home/hakubishin3/anaconda3/lib/python3.6/site-packages/upsilon/extract_features/extract_features.py:61: UserWarning: The number of data points are less than 80.
  % min_n_data)


OrderedDict([('amplitude', 386.304197032748),
             ('cusum', 0.20611636592760962),
             ('eta', 1.6578265891873614),
             ('hl_amp_ratio', 1.3300739250634674),
             ('kurtosis', -1.5599369998179708),
             ('n_points', 57),
             ('period', 0.3244800372208426),
             ('period_SNR', 23.889243076602337),
             ('period_log10FAP', -6.048343754404527),
             ('period_uncertainty', 8.051614322582279e-05),
             ('phase_cusum', 0.2982932499364303),
             ('phase_eta', 0.3547877159445853),
             ('phi21', 2.1748830759803566),
             ('phi31', 3.045055087059272),
             ('quartile31', 611.040497),
             ('r21', 0.044728513618950985),
             ('r31', 0.039216077576528695),
             ('shapiro_w', 0.8778709173202515),
             ('skewness', 0.12194754284259278),
             ('slope_per10', -0.0003577323100311799),
             ('slope_per90', 0.0003078394874940904),
            

In [37]:
import warnings
warnings.filterwarnings("ignore")

result_list = []
for object_id in tqdm_notebook(train_ts.object_id.unique(), ascii=True):
    for passband in train_ts.passband.unique():
        sample_data = train_ts.query('object_id == @object_id & passband == @passband')
        e_features = upsilon.ExtractFeatures(
            sample_data["mjd"], sample_data["flux"], sample_data["flux_err"],
            n_threads=8)
        e_features.run()
        features = e_features.get_features()
        result = [object_id, passband] + list(features.values())
        result_list.append(result)

TypeError: Improper input: N=7 must not exceed M=6

In [38]:
colnames = ['object_id', 'passband'] + list(features.keys())
results = pd.DataFrame(result_list, columns=colnames)
results.set_index(['object_id', 'passband']).unstack()

amplitude                                                      \
passband             0            1            2            3            4   
object_id                                                                    
615         112.605911   882.445501   665.610213   484.054636   409.720682   
713          10.692238     7.539003     6.645727     8.576184     6.606117   
730           1.418107     1.320970    13.615920    20.830331    28.526381   
745           5.059816    46.389607    60.421121    64.871561    59.286871   
1124          1.657526    17.218355    53.725319    73.560465    79.426832   
1227          4.810036     1.282579     1.166443     1.787652     2.815051   
1598         96.120389   615.946085   542.759685   435.292557   328.485642   
1632          1.586598     1.249825     1.194724     2.105944     8.397496   
1920          6.468356    44.375708    66.032552    72.263615    64.236281   
1926          3.520881     1.436208     4.759795     1.971907     4.067816   
2072          1.486205     8.419175    14.613440     7.617870     8.510948   
2103         17.101273    51.003748    50.965965    35.481083    39.009918   
2300          1.319694     1.369349     0.960327    10.135821    12.192997   
2330          1.387505     1.090775     2.014110     6.341563     4.758267   
2624          7.424357     4.895804     7.019414    15.330492   172.630686   
2677          2.000265    16.013160    24.337333    21.613219    15.945199   
2922          2.387433    16.335752    88.112508   143.583476   144.906347   
3041          1.415344     1.498757    10.807666     9.609176    18.822702   
3285          6.801181    30.779127    44.900443    31.333011    34.289028   
3423          2.004065     1.456632     1.874360     7.139252    14.269013   
3489          6.748211     9.918222    10.121703     9.738237    12.551399   
3910          1.962545     3.240741     9.510318    23.028586    30.259083   
4088         44.104127    48.789775    43.501200    50.911681    51.681193   
4132          1.458064    18.228696    28.224761    53.383861    14.292450   
4171          1.671731     9.896986    13.194804     8.433515     4.493210   
4173         52.377393    79.330293    64.253299    51.629622    47.034321   
4220         96.540267    84.794537   104.162242   117.075443   122.709047   
4389          1.918548     9.972594    30.196516    46.676884    63.573766   
4595         11.594403     8.848400    27.015691    31.872709    35.553124   
4819         32.210740    17.490215    32.755176   163.491897   137.269329   
...                ...          ...          ...          ...          ...   
1497514      17.177488    31.683759    22.711357    68.768355    92.843839   
1516088      22.800538     9.124201    26.324421    10.309909    34.470509   
1535056      38.483551    27.188932     5.404434     6.177065    14.059240   
1557442       7.470276   166.155857   288.495954    74.554723    75.653651   
1563124     567.052822   225.217137   232.249830   162.208838    67.719381   
1573501     202.815273    29.638882    47.639955    32.100241   224.788960   
1594469      55.142789    80.252613    47.682128    73.247394    69.592601   
1597464      47.542472    17.805908   171.273652    12.486121    16.941365   
1624308       8.034801   408.721605     8.306900   184.814056    14.032320   
1634735      11.836235    10.365472    34.705832    60.534362   130.605668   
1658030     105.426105    86.827279   107.774600   102.305529   108.138380   
1744454      18.975286    76.862191    19.252137    26.483225    36.911190   
1797009      55.430494   223.244312     6.159608   117.795387    52.857630   
1838516      16.844713    99.940214     9.806881    18.876446    15.948996   
1854631     135.407022   998.286773   190.769891   334.971482  1478.813059   
1879121      36.148723    70.473003   254.171521   180.905886   171.017945   
1904697      87.855427   167.820798   258.596278   199.178938   155.651429   
1919887      13.156421    48.736412